# "Web Scraping Assignment-3"

In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
#Importing requests
import requests

# importing regex
import re

# Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
# connect to webdriver
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(2)

# to get url
url = "https://www.amazon.in/"
driver.get(url)

time.sleep(2)
# input the keywords that user wants to search
user_input = input('Enter the product you want to search : ')

Enter the product you want to search : guitars


In [3]:
# to Locate searc_bar by id
search_bar = driver.find_element(By.ID,"twotabsearchtextbox") 
# to clear the search_bar
search_bar.clear() 
# to send user input to search bar
search_bar.send_keys(user_input)
 # to Locate search_button by xpath
search_button = driver.find_element(By.XPATH,'//div[@class="nav-search-submit nav-sprite"]/span/input')      
search_button.click()              

# Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
#collecting all the Product URLS
urls = []
for i in range(0,3):
    Page_urls=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')#collecting urls of all the guitar
    for i in Page_urls:
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]') 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [7]:
#Making empty lists and scraping the required spots
Product_name = []
Brand_name= []
Price = []
Return = []
Expected_Delivery = []
Availability = []

#Start with for loop
for i in urls:
    driver.get(i)
    time.sleep(2)
    
    
    #Scraping product name
    try:
        prod=driver.find_element(By.XPATH,'//span[@class="a-size-base-plus a-color-base a-text-normal"]')
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
        
        
    #Scraping brand name
    try:
        brand=driver.find_element(By.XPATH,'//div[@id="bylineInfo_feature_div"]/div/a')
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
        
    #Scraping Price
    try:
        pri=driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')  
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
        
    #Scraping Return/Exchange
    try:
        ret=driver.find_element(By.XPATH,'//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
        
     
    #Scraping  Expected_Delivery
    try:
        delivary=driver.find_element(By.XPATH,'//span[@class="a-color-base a-text-bold"]')
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
        
    #Scraping Availability
    try:
        avai=driver.find_element(By.XPATH,'//div[@id="availability"]')
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")

        
# To make dataframe
Guitar=pd.DataFrame({})
Guitar['Name'] = Product_name
Guitar['Brand'] = Brand_name
Guitar['Price'] = Price
Guitar['Return'] = Return
Guitar['Expected_Delivery'] = Expected_Delivery
Guitar['Availability'] = Availability
Guitar['Urls'] = urls

In [8]:
#Print dataframe
Guitar

,Name,Brand,Price,Return,Expected_Delivery,Availability,Urls
0,-,-,-,-,-,-,https://www.amazon.in/gp/slredirect/picassoRed...
1,-,Visit the Kadence Store,,7 Days Replacement,-,,https://www.amazon.in/gp/slredirect/picassoRed...
2,-,Brand: Exegi Enterprise,999,7 Days Returnable,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
3,-,Brand: KETOSTICS,"4,199",7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
4,-,Visit the JUAREZ Store,,7 Days Replacement,-,In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
158,-,Brand: Gavison,"3,750",7 Days Replacement,-,In stock.,https://www.amazon.in/Gavison-Venus-Black-Rose...
159,-,Visit the YAMAHA Store,699,7 Days Replacement,-,,https://www.amazon.in/Yamaha-Acoustic-Guitar-T...
160,-,Brand: Fender,"7,586",7 Days Replacement,-,In stock.,https://www.amazon.in/Fender-SA-150C-dreadnoug...
161,-,Brand: Ibanez,"22,000",7 Days Replacement,-,In stock.,https://www.amazon.in/Ibanez-GRG170DX-Electric...


In [9]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="b64799022dcecefcb552cb783fba4c80")>>

# Q3.Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [35]:
# Connecting to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
# Getting the webpage of mentioned url 
url = "https://images.google.com/"
#Creating empty list 
urls = []    #empty list
data = []    #empty list
search_item = ["fruits", "cars", "Machine Learning", "Guitar", "Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.TAG_NAME,"input") 
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(str(item))
    
    # Clicking on search button
    search_button = driver.find_element(By.XPATH,'//button[@class="Tg7LZd"]').click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(50):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 50:
        break
    print("Downloading {0} of {1} images" .format(i, 50))
    response = requests.get(urls[i])
    file = open(r"C:\Users\akash\fliprobo\google pics"+str(i)+".jpg","wb")
    file.write(response.content)

In [36]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="ac7f2221ff44c0b58f41508aba400b9d")>>

# Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [37]:
# connect to wed browser
driver = webdriver.Chrome(r"chromedriver.exe")

# To get the web page 
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)

In [38]:
# to close login pop up
driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _2doB4z']").click()

In [39]:
# to enter the item name which user wants to search
item = input(" Enter the name of Smartphone that has to be searched : ")

#giving input key word to search bar
serch_bar = driver.find_element(By.XPATH,"//div[@class='_3OO5Xc']//input")
serch_bar.send_keys(item)

# to click search button
srch_btn = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

 Enter the name of Smartphone that has to be searched : ONEPLUS NORD


In [40]:
# To get  urls of phones on 1st page
page1_urls = []
urls = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [41]:
len(page1_urls)

24

In [42]:
# to make emoty list for all features to be scraped
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

In [43]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element(By.XPATH,'//li[@class="_21lJbe"]')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
        
      #Scraping RAM data
    try:
        ram_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
    
    #Scraping Display size data 
    try:
        disp_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
     

    
    #Scraping battery capacity data
    try:
        if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
     #Scraping Price data
    try:
        price_tags = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')
        
        
    

Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-black-dusk-128-gb/p/itm7acae55b999e6?pid=MOBGHH9FVUHCFHTY&lid=LSTMOBGHH9FVUHCFHTYVDUFZS&marketplace=FLIPKART&q=ONEPLUS+NORD&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=8f657abf-5b53-41d6-9fc7-8260467b9acb.MOBGHH9FVUHCFHTY.SEARCH&ppt=hp&ppn=homepage&ssid=d5rsiamm1s0000001662302678533&qH=93fc5fde505957dd
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-blue-tide-128-gb/p/itm7acae55b999e6?pid=MOBGHHXQCZQVEQDG&lid=LSTMOBGHHXQCZQVEQDG02RUHD&marketplace=FLIPKART&q=ONEPLUS+NORD&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=8f657abf-5b53-41d6-9fc7-8260467b9acb.MOBGHHXQCZQVEQDG.SEARCH&ppt=hp&ppn=homepage&ssid=d5rsiamm1s0000001662302678533&qH=93fc5fde505957dd
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-blue-tide-128-gb/p/itm7acae55b999e6?pid=MOBGDY3QRGSPCZW3&lid=LSTMOBGDY3QRGSPCZW3FRYPDH&marketplace=FLIPKART&q=ONEPLUS+NORD&store=t

Scraping URL =  https://www.flipkart.com/oneplus-nord-gray-onyx-128-gb/p/itm49f817b591982?pid=MOBFUE5HW4XUUGBN&lid=LSTMOBFUE5HW4XUUGBNMZB8TR&marketplace=FLIPKART&q=ONEPLUS+NORD&store=tyy%2F4io&srno=s_1_22&otracker=search&otracker1=search&fm=organic&iid=8f657abf-5b53-41d6-9fc7-8260467b9acb.MOBFUE5HW4XUUGBN.SEARCH&ppt=hp&ppn=homepage&ssid=d5rsiamm1s0000001662302678533&qH=93fc5fde505957dd
Scraping URL =  https://www.flipkart.com/oneplus-nord-blue-marble-128-gb/p/itm49f817b591982?pid=MOBFUE5JWYDZX6HX&lid=LSTMOBFUE5JWYDZX6HXJZSHIV&marketplace=FLIPKART&q=ONEPLUS+NORD&store=tyy%2F4io&srno=s_1_23&otracker=search&otracker1=search&fm=organic&iid=8f657abf-5b53-41d6-9fc7-8260467b9acb.MOBFUE5JWYDZX6HX.SEARCH&ppt=hp&ppn=homepage&ssid=d5rsiamm1s0000001662302678533&qH=93fc5fde505957dd
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-black-dusk-128-gb/p/itm7acae55b999e6?pid=MOBGHHXZYSZ4UWYG&lid=LSTMOBGHHXZYSZ4UWYGETWXU3&marketplace=FLIPKART&q=ONEPLUS+NORD&store=tyy%2F4io&srno=s_1_24&o

In [44]:
#Checking lengths of all scraped data
print(len(Smartphones["Brand"]), len(Smartphones["Phone name"]), len(Smartphones["Colour"]), len(Smartphones["RAM"]), len(Smartphones["Storage(ROM)"]))

24 24 24 24 24


In [45]:
#Framing the data
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,URL
0,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,518",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,529",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,8 GB,128 GB,Primary Camera,-,16.74 cm (6.59 inch),-,"₹21,998",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,"Handset,SIM Tray Ejector, Adapter, Phone Case,...",Black Dusk,6 GB,128 GB,Primary Camera,-,16.74 cm (6.59 inch),-,"₹19,329",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,"Mobile, Adapter, Phone Case, SIM Tray Ejector,...",Gray Shadow,12 GB,256 GB,Primary Camera,-,17.02 cm (6.7 inch),-,"₹33,615",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
5,OnePlus,"Mobile, Adapter, Phone Case, SIM Tray Ejector,...",Gray Shadow,8 GB,128 GB,Primary & Secondary Camera,-,17.02 cm (6.7 inch),-,"₹28,749",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
6,OnePlus,"Mobile, Adapter, Phone Case, SIM Tray Ejector,...",Jade Fog,12 GB,256 GB,Primary Camera,-,17.02 cm (6.7 inch),-,"₹33,999",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
7,OnePlus,"Handset,SIM Tray Ejector, Adapter, Phone Case,...",Blue Tide,6 GB,128 GB,16MP Front Camera,-,16.74 cm (6.59 inch),-,"₹19,989",https://www.flipkart.com/oneplus-nord-ce-2-lit...
8,OnePlus,"Mobile, Adapter, Phone Case, SIM Tray Ejector,...",Jade Fog,12 GB,256 GB,Primary & Secondary Camera,-,17.02 cm (6.7 inch),-,"₹33,790",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
9,OnePlus,"Mobile, Adapter, Phone Case, SIM Tray Ejector,...",Gray Shadow,12 GB,256 GB,Primary & Secondary Camera,-,17.02 cm (6.7 inch),-,"₹33,900",https://www.flipkart.com/oneplus-nord-2t-5g-gr...


In [46]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="96a94c6fd0e7c989aaba4f8f10503257")>>

# Q5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [49]:
# to get web browser
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(2)

# opening google maps
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)


#Sending keyword for seach bar and search button
city = input('Enter City name that has to be searched : ')
search_bar = driver.find_element(By.ID,"searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)
search_bar.send_keys(city)                                                     
search_btn = driver.find_element(By.ID,"searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : Patna
URL Extracted:  https://www.google.co.in/maps/place/Patna,+Bihar/@20.9880135,82.7525294,5z/data=!4m5!3m4!1s0x39f29937c52d4f05:0x831a0e05f607b270!8m2!3d25.5940947!4d85.1375645
Latitude = 20.9880135, Longitude = 82.7525294


In [50]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="7db6f69db9d5abd5b91abe5e4191dd6c")>>

# Q6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [52]:
# to get web browser
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(2)

#to get url
driver.get('https://trak.in/')

#to get xpath for funding deals
fund_button = driver.find_element(By.XPATH,'//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

In [55]:
# Empty lists
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


for i in range(54,57):
    # Date
    dt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
      # Investor
    inv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df


,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [56]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="c6156b1e940ec6dc33332f8b8894ea48")>>

# Q7.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [2]:
# Connecting to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

In [3]:
# opening the url digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [4]:
#searching for best laptop
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [5]:
#Creating empty lists
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [6]:
# Scraping the data of laptop names
laptop_name= driver.find_elements(By.XPATH,"//div[@class='right-container']/div/a/h3")
for name in laptop_name:
    Laptop_Name.append(name.text)
    
    
#Scraping the data of operating system
try:
    op_sys = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display of the laptop
try:
    display= driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass

#Scraping data of memory
try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass

#Scraping data of weight
try:
    weight = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


#Scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    graph = driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    price = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [7]:
print(len(Laptop_Name),len(Operating_sys),len(Display),len(Processor),len(Memory),len(Weight),len(Dimensions),len(Graph_proc),len(Price))

10 10 10 10 10 10 10 10 10


In [8]:
data=list(zip(Laptop_Name, Operating_sys, Display, Processor, Memory, Weight, Dimensions, Graph_proc, Price))

In [9]:
df=pd.DataFrame(data,columns=['Laptop_Name','Operating_System','Display','Processor','Memory','Weight','Dimension','Graph_Proc','Price'])

In [10]:
df

,Laptop_Name,Operating_System,Display,Processor,Memory,Weight,Dimension,Graph_Proc,Price
0,MSI TITAN GT77-12UHS,WINDOWS 11 HOME OS,"17.3"" (3840 X 2160) DISPLAY",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,N/A
1,ALIENWARE X17 R2,WINDOWS 11 HOME OS,"17.3"" (1920X1080) DISPLAY",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,"16 GB DDR6 NVIDIA GeForce RTX 3080 Ti, Graphic...",399.23 x 299.57 x 14.75 mm dimension & 3.02 kg...,"16 GB DDR6 NVIDIA GeForce RTX 3080 Ti, Graphic...","Rs 389,990"
2,ACER PREDATOR TRITON 500 SE PT516-52S,WINDOWS 11 HOME OS,"16"" (2560 X 1600) DISPLAY",Windows 11 Home,32 GB DDR5 RAM & 2 TB SSD,8 GB DDR6 NVIDIA GeForce RTXTM 3070 Ti Graphic...,358 x 262 x 19.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTXTM 3070 Ti Graphic...,"Rs 300,000"
3,OMEN BY HP (16-B1371TX),WINDOWS 11 HOME OS,"16.1"" (2560 X 1440) DISPLAY",Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,369 x 248 x 23 mm dimension & 2.32 kg weight,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 194,000"
4,ACER PREDATOR HELIOS 300 AN515-45 (NH.QBRSI.0,WINDOWS 11 HOME OS,"15.6"" (2560 X 1440) DISPLAY",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,363 x 255 x 23.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 172,999"
5,MSI DELTA 15 (A5EFK-083IN),WINDOWS 11 HOME OS,"15.6"" (1920 X 1080) DISPLAY",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,10 GB DDR6 AMD Radeon RX 6700M Graphics card,357 x 247 x 19 mm dimension & 1.9 kg weight,10 GB DDR6 AMD Radeon RX 6700M Graphics card,"Rs 188,990"
6,OMEN BY HP (16-C0141AX),WINDOWS 11 HOME OS,"16.1"" (2560 X 1440) DISPLAY",Windows 11 Home,16 GB DDR4 RAM & 1 TB NVMe,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,"Rs 129,899"
7,LENOVO LEGION 5I PRO (82RF00MGIN),WINDOWS 11 HOME OS,"16"" (2560 X 1600) DISPLAY",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,359.9 x 264.4 x 19.9 mm dimension & 2.5 kg weight,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 230,890"
8,ALIENWARE M15 R5 RYZEN EDITION ICC-C780001WIN,WINDOWS 11 HOME OS,"15.6"" (1920 X 1080) DISPLAY",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 144,990"
9,LENOVO SLIM 7 GEN 6 (82K8002JIN),WINDOWS 11 HOME OS,15.6 MP | NA DISPLAY,Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,356 x 252 x 16 mm dimension & 1.9 kg weight,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,"Rs 131,990"


# Q8.Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [2]:
# Connecting to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

In [7]:
#Getting the specified url
url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)
time.sleep(3)

In [8]:
#clicking on the explore button
button = driver.find_element(By.XPATH,'//div[@class="_69hVhdY4"]')
button.click()
time.sleep(1)

In [9]:
#select billionaire  
bill = driver.find_element(By.XPATH,'//li[@class="TjJgrPSg cD45ib6e primary"]')
bill.click()
time.sleep(1)

In [10]:
#select world billionaire  
world_bill= driver.find_element(By.XPATH,'//li[@class="TjJgrPSg _2bNo56RE secondary"]')
world_bill.click()
time.sleep(1)

In [11]:
#Scraping required data from the web page
#Creatng empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []

In [14]:
while(True):
    #Scraping the data of rank of the billionaires
    rank_tags= driver.find_elements(By.XPATH,'//div[@class="rank"]')
    for rank in rank_tags:
        Rank.append(rank.text)
    time.sleep(1)
    
    
    #Scraping the data of names of the billionaires
    name_tags= driver.find_elements(By.XPATH,'//div[@class="personName"]/div')
    for name in name_tags:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    #Scraping data of age of the billionaires
    age_tags= driver.find_elements(By.XPATH,'//div[@class="age"]/div')
    for age in age_tags:
        Age.append(age.text)   
    time.sleep(1)
    
    
    #Scraping data of citizenship of the billionaires
    cit_tags= driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
    for cit in cit_tags:
        citizenship.append(cit.text)
    time.sleep(1)
    
    
    #Scraping data of source of income of the billionaires
    sour_tags= driver.find_elements(By.XPATH,'//div[@class="source"]')
    for sour in sour_tags:
        Source.append(sour.text)    
    time.sleep(1)
    
    
    #Scraping data of Industry of the billionaires
    ind_tags= driver.find_elements(By.XPATH,'//div[@class="category"]//div')
    for ind in ind_tags:
        industry.append(ind.text)
        
    
    #scraping data of net_worth of billionaires
    net_tags= driver.find_elements(By.XPATH,'//div[@class="netWorth"]//div[1]')
    for net in net_tags:
        total_net_worth.append(net.text)
    time.sleep(1)
    
    
    #Clicking on next button
    try:
        next_button = driver.find_element(By.XPATH,"//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
#Scraping data of net worth
Net_Worth = []
for i in range(0,len(total_net_worth),2):
    Net_Worth.append(total_net_worth[i])

In [15]:
#Framing data
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,,United States,"Tesla, SpaceX",
1,2.,Jeff Bezos,$171 B,,United States,Amazon,
2,3.,Bernard Arnault & family,$158 B,,France,LVMH,
3,4.,Bill Gates,$129 B,,United States,Microsoft,
4,5.,Warren Buffett,$118 B,,United States,Berkshire Hathaway,
...,...,...,...,...,...,...,...
2663,2578.,Zhang Yuqiang,$1 B,,China,Fiberglass,
2664,2578.,Zhou Ruxin,$1 B,,China,Navigation,
2665,2578.,Wen Zhou & family,$1 B,,China,chemicals,
2666,2578.,Zhou Yifeng & family,$1 B,,China,liquefied petroleum gas,


In [16]:
#Saving dataset in csv
Billionaires.to_csv("Forbes_Billionaries.csv")

# Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [28]:
# to get web browser
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(2)
# to get url
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [30]:
# input the keywords that user wants to search
user_input = input('Enter the video you want to search : ')

#finding element for search bar
search_bar = driver.find_element(By.XPATH,'//input[@id="search"]')
search_bar.send_keys(user_input)
time.sleep(1)

Enter the video you want to search : RRR


In [32]:
#clicking on search button
search_btn = driver.find_element(By.ID,"search-icon-legacy")  
search_btn.click()
time.sleep(1)

In [33]:
#clicking on first video
link_click = driver.find_element(By.XPATH,"//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

# 10000 time we scroll down to generate more Comments
for _ in range(10000):
    driver.execute_script("window.scrollBy(0,10000)")

In [35]:
# empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

# to scrape comments
cm_tags = driver.find_elements(By.ID,"content-text")
for cm in cm_tags:
    if cm.text is None:
        comments.append("--")
    else:
        comments.append(cm.text)
time.sleep(5)
comments[:500]

['Goosebumps Performance By Ntr & Charan ',
 'This should be played in schools it shows how poorly any race besides white was treated then and now. Fantastic movie 10/10 recommend.',
 "People still thinks that Hollywood is the benchmark of achievement. I don't agree, i say India and indien cinema is . Love from Montréal 🇨🇦",
 'love India 🇮🇳  from Somalia 🇸🇴',
 'Completely awesome. My heart would explode after 30 seconds of that ',
 'THIS IS THE F*CKING BEST MOVIE IN THE WORLD!! Im not Indian but these movie shows all the powar of the world outside of America or Europe!! NOT SALSA, NOT FLAMENCO MY BROTHER!!',
 'Theatrical experience of this song is unbelievable. The choreography is mind blowing.',
 "First experience with Indian cinema, and it's amazing. The entire movie was incredible, but this scene really blew me away. The energy and pure joy that NTR and Ram Charan put into this scene is just awesome. And when Ram gave the British guy two thumbs up with that big smile on his face, lo

In [36]:
len(comments)

3019

In [37]:
# to scrape time when comment was posted
tm_tags = driver.find_elements(By.XPATH,"//a[contains(text(),'ago')]")
for tm in tm_tags:
    if tm.text is None:
        Time.append("--")
    else:
        Time.append(tm.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
comment_time[:500]

# to scrape comment likes
like_tags = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for like in like_tags:
    if like.text is None:
        Likes.append("--")
    else:
        Likes.append(like.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
Likes[:500]    

['',
 '6.2K',
 '',
 '21',
 '',
 '19',
 '',
 '32',
 '',
 '274',
 '',
 '25',
 '',
 '8.2K',
 '',
 '1.9K',
 '',
 '243',
 '',
 '10',
 '',
 '586',
 '',
 '159',
 '',
 '211',
 '',
 '120',
 '',
 '182',
 '',
 '8',
 '',
 '226',
 '',
 '4',
 '',
 '21K',
 '',
 '458',
 '',
 '199',
 '',
 '7',
 '',
 '369',
 '',
 '54',
 '',
 '755',
 '',
 '72',
 '',
 '50',
 '',
 '279',
 '',
 '5',
 '',
 '438',
 '',
 '142',
 '',
 '755',
 '',
 '130',
 '',
 '72',
 '',
 '47',
 '',
 '392',
 '',
 '17',
 '',
 '166',
 '',
 '7',
 '',
 '346',
 '',
 '52',
 '',
 '137',
 '',
 '141',
 '',
 '140',
 '',
 '26',
 '',
 '858',
 '',
 '5',
 '',
 '426',
 '',
 '22',
 '',
 '67',
 '',
 '14',
 '',
 '42',
 '',
 '4',
 '',
 '373',
 '',
 '58',
 '',
 '62',
 '',
 '18',
 '',
 '32',
 '',
 '8',
 '',
 '1.4K',
 '',
 '46',
 '',
 '72',
 '',
 '6',
 '',
 '400',
 '',
 '15',
 '',
 '3.3K',
 '',
 '46',
 '',
 '82',
 '',
 '15',
 '',
 '32',
 '',
 '36',
 '',
 '196',
 '',
 '24',
 '',
 '324',
 '',
 '4',
 '',
 '193',
 '',
 '6',
 '',
 '54',
 '',
 '180',
 '',
 '9',
 '',
 '4',

In [38]:
#Creating dataframe
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments[:500]
Youtube['Comment_time'] = comment_time[:500]
Youtube['Comment upvotes'] = No_of_Likes[:500]

#Printing dataframe
Youtube

,Comments,Comment_time,Comment upvotes
0,Goosebumps Performance By Ntr & Charan,4 months ago,6.2K
1,This should be played in schools it shows how ...,7 days ago (edited),21
2,People still thinks that Hollywood is the benc...,13 days ago,19
3,love India 🇮🇳 from Somalia 🇸🇴,4 months ago,32
4,Completely awesome. My heart would explode aft...,1 month ago,274
...,...,...,...
495,Awesome,2 months ago,2
496,The power of tollywood,2 months ago (edited),1
497,"I saw this movie ten times in my laptop,phone ...",2 months ago,2
498,చరిత్ర సృష్టించలన్నా మనమే...\nదాన్ని తిరిగి రా...,2 months ago,47


# Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [15]:
# to get web browser
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(3)

# to get url
url = "https://www.hostelworld.com/"
driver.get(url)

In [16]:
#to scrape location search bar
search_loc = driver.find_element(By.XPATH,'//input[@class="search-input"]')
search_loc.send_keys("London")
time.sleep(2)

In [17]:
#select london
london = driver.find_element(By.XPATH,'//div[@class="label"]')
london.click()
time.sleep(2)

In [18]:
# do click in search button
search_btn = driver.find_element(By.ID,"search-button")
search_btn.click()

In [19]:
#empty lists
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url = []

In [25]:
for i in driver.find_elements(By.XPATH,"//div[@class = 'pagination-item pagination-current']"):
    i.click()
    time.sleep(4)
    #to get hostel name
    try:
        name = driver.find_elements(By.XPATH,"//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
    #to get distance from city centre
    
    try:
        dist = driver.find_elements(By.XPATH,"//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements(By.XPATH,"//div[@class='prices-col']"):
    #to get privates from price
        try:
            pvt_price = driver.find_element(By.XPATH,"//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
    #to get  dorms from price
    for i in driver.find_elements(By.XPATH,"//div[@class='prices-col']"):
        try:
            dorms = driver.find_element(By.XPATH,"//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
            #to get facilities
    try:
        fac1 = driver.find_elements(By.XPATH,"//div[@class='has-wifi']")
        fac2 = driver.find_elements(By.XPATH,"//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
    #to fetch url of each hostel
    p_url = driver.find_elements(By.XPATH,"//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    #to click on show more button for description
    try:
        driver.find_element(By.XPATH,'//button[@class="button primary small full-width"]').click()
        time.sleep(5)
    except NoSuchElementException:
        pass
    #to get ratings
    try:
        rat = driver.find_element(By.XPATH,"//div[@class='score orange big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
    # to get total reviews
        
    try:
        rws = driver.find_element(By.XPATH,"//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        #to get  overall review
    try:
        overall_rw = driver.find_element(By.XPATH,"//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
    #to get property description 
    try:
        disc = driver.find_element(By.XPATH,"//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')
                

In [30]:
#to make dataframe
dff = pd.DataFrame({})
dff['Hostel_Name'] = hostel_name
dff['Distance fron city centre'] = distance
dff['Ratings'] = rating
dff['Total_reviews'] = reviews
dff['Overall Reviews'] = over_all
dff['Privates from price'] = pvt_prices
dff['Dorms from price'] = dorms_price
dff['Facilities'] = facilities[:60]
dff['Description'] = description

In [31]:
#Printing data frame
dff

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Facilities,Description
0,Palmers Lodge - Swiss Cottage,6.5km from city centre,8.7,15568,Fabulous,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","40 College Crescent, Swiss Cottage, London, En..."
1,Selina Camden,5.5km from city centre,8.6,36,Fabulous,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","89 Chalk Farm Rd, Chalk Farm, London, England"
2,St Christopher's Village,1.8km from city centre,8.1,11487,Fabulous,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","165 Borough High Street, London, England"
3,Wombat's City Hostel London,3.6km from city centre,9.0,14015,Superb,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","7 Dock Street, London, England"
4,Generator London,3km from city centre,7.8,7139,Very Good,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","Compton Place, Off 37 Tavistock Place, WC1, Lo..."
5,Astor Museum Inn,2.2km from city centre,8.2,9048,Fabulous,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","27 Montague Street, Bloomsbury, London, England"
6,Urbany Hostel London,5.4km from city centre,9.3,454,Superb,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","48-49 Princes Square, Paddington, London, England"
7,Safestay London Elephant & Castle,1.7km from city centre,7.0,4476,Very Good,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","144-152 Walworth Road, London, England"
8,Pickwick Hall,2.3km from city centre,8.5,2447,Fabulous,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","7 Bedford Place, Bloomsbury, London, England"
9,No.8 Willesden Hostel London,10km from city centre,6.6,4831,Good,Rs19731,Rs1922,"Free WiFi, Follows Covid-19 sanitation guidance","305 High Rd, Willesden, London, England"


In [32]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="9903cfffd5a089e2f5fc6c98c8b037cb")>>